# Analyse étendue de la concentration du pouvoir de vote

Ce notebook calcule plusieurs indicateurs de concentration du pouvoir pour chaque fichier de vote. Les mesures incluent :
- Coefficient de Gini
- Entropie de Shannon
- Indice de Theil
- Ratio du vote contrôlé par le top 1 %, 10 % et 50 %
- Indice de Herfindahl-Hirschman (HHI)

Les résultats sont ensuite corrélés avec le coefficient de Gini pour vérifier la cohérence des mesures. Seuls les fichiers contenant plus de **10 votants** sont pris en compte.


In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

# Chemin vers le dossier contenant les fichiers de vote (CSV)
# Chaque fichier doit posséder une colonne 'voting_power' représentant la force de vote de chaque électeur
VOTES_DIR = 'path_to_vote_files'

records = []

for filename in os.listdir(VOTES_DIR):
    if not filename.endswith('.csv'):
        continue
    file_path = os.path.join(VOTES_DIR, filename)
    df = pd.read_csv(file_path)
    if 'voting_power' not in df.columns:
        continue
    voting_power = df['voting_power'].dropna().astype(float)
    n = len(voting_power)
    if n <= 10:
        continue
    total_power = voting_power.sum()
    shares = voting_power / total_power

    def gini(array):
        array = np.sort(array)
        index = np.arange(1, array.size + 1)
        return (np.sum((2 * index - array.size - 1) * array)) / (array.size * array.sum())

    gini_coeff = gini(voting_power.values)

    shannon_entropy = -(shares * np.log(shares + 1e-12)).sum()

    mean_power = voting_power.mean()
    theil = (voting_power / mean_power * np.log((voting_power + 1e-12) / mean_power)).mean()

    hhi = (shares ** 2).sum()

    df_sorted = voting_power.sort_values(ascending=False)
    top1 = df_sorted.iloc[:max(1, int(0.01 * n))].sum() / total_power
    top10 = df_sorted.iloc[:max(1, int(0.10 * n))].sum() / total_power
    top50 = df_sorted.iloc[:max(1, int(0.50 * n))].sum() / total_power

    records.append({
        'proposal': filename,
        'gini': gini_coeff,
        'shannon_entropy': shannon_entropy,
        'theil': theil,
        'hhi': hhi,
        'top1_share': top1,
        'top10_share': top10,
        'top50_share': top50,
    })


df_metrics = pd.DataFrame(records)
df_metrics.head()


In [ ]:
# Matrice de corrélation et p-values
metrics = ['gini', 'shannon_entropy', 'theil', 'hhi', 'top1_share', 'top10_share', 'top50_share']

corr_matrix = df_metrics[metrics].corr()

pval_matrix = pd.DataFrame(index=metrics, columns=metrics)
for i in metrics:
    for j in metrics:
        corr, pval = pearsonr(df_metrics[i], df_metrics[j])
        pval_matrix.loc[i, j] = pval

signif_levels = {0.001: '***', 0.01: '**', 0.05: '*'}

def format_corr(value, p):
    for thresh, star in signif_levels.items():
        if p < thresh:
            return f"{value:.2f}{star}"
    return f"{value:.2f}"

corr_with_sig = corr_matrix.copy().astype(str)
for i in metrics:
    for j in metrics:
        corr_with_sig.loc[i, j] = format_corr(corr_matrix.loc[i, j], pval_matrix.loc[i, j])

corr_with_sig


In [ ]:
# Sauvegarde éventuelle des résultats
# df_metrics.to_csv('voting_power_metrics.csv', index=False)
